This code is intended to calculate the DICE coefficients for the images produced by the different segmentation models under considerations. The inputs for this code are the .labeling files produced using ImageJ/Fiji's Labkit plugin as well as the .npy files produced by Scikit-image, Cellpose and CellSeg

In [27]:
import os
from PIL import Image
import numpy as np
import pandas as pd

def load_image_as_mask(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    mask = np.array(img)
    return (mask > 128).astype(bool)  # Binarize assuming white=foreground, black=background

def dice_coefficient_from_masks(ground_truth_mask, predicted_mask):
    ground_truth_mask = ground_truth_mask.astype(bool)
    predicted_mask = predicted_mask.astype(bool)

    if predicted_mask.shape != ground_truth_mask.shape:
        # Resize predicted mask to match ground truth
        predicted_mask = np.array(
            Image.fromarray(predicted_mask).resize(ground_truth_mask.shape[::-1], Image.NEAREST)
        )

    intersection = np.logical_and(predicted_mask, ground_truth_mask).sum()
    total_pixels = predicted_mask.sum() + ground_truth_mask.sum()
    dice = (2.0 * intersection) / total_pixels if total_pixels > 0 else 1.0
    return dice

def find_predicted_mask(predicted_folder, filename_base):
    # Check for various common image file extensions
    for ext in ['.tif', '.jpg', '.png']:
        predicted_path = os.path.join(predicted_folder, filename_base + ext)
        if os.path.exists(predicted_path):
            return predicted_path
    return None

def calculate_dice_and_save_to_excel(ground_truth_folder, predicted_folder, output_excel):
    dice_scores = []

    for filename in os.listdir(ground_truth_folder):
        ground_truth_path = os.path.join(ground_truth_folder, filename)
        filename_base = filename.rsplit('.', 1)[0]  # Base name without extension

        # Load the ground truth mask based on file extension
        if filename.endswith(('.labeling', '.jpg', '.tif')):
            ground_truth_mask = load_image_as_mask(ground_truth_path)
        else:
            print(f"Unsupported file format for {filename}. Skipping...")
            continue

        # Find the predicted mask file with any common image extension
        predicted_path = find_predicted_mask(predicted_folder, filename_base)

        if predicted_path:
            try:
                predicted_mask = load_image_as_mask(predicted_path)

                # Calculate Dice coefficient
                dice_score = dice_coefficient_from_masks(ground_truth_mask, predicted_mask)
                dice_scores.append({'Filename': filename, 'Dice Coefficient': dice_score})
                print(f"Dice Coefficient for {filename}: {dice_score}")

            except Exception as e:
                print(f"Error loading predicted mask for {filename}: {e}")
                continue
        else:
            print(f"Predicted mask for {filename} not found in {predicted_folder}")

    # Save results to specified output path
    dice_df = pd.DataFrame(dice_scores)
    dice_df.to_excel(output_excel, index=False)
    print(f"\nDice coefficients saved to {output_excel}")

# Example usage
ground_truth_folder = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Manual Segmentation (Labkit) - CellSeg'
predicted_folder = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\CellSeg\Gray Scale'
output_excel = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\DICE\cellseg_gray.xlsx'

calculate_dice_and_save_to_excel(ground_truth_folder, predicted_folder, output_excel)


Dice Coefficient for D0-T25-10x1.tif: 0.0
Dice Coefficient for D0-T25-20x1.tif: 0.0
Dice Coefficient for D0-T25-40x1.tif: 1.0
Dice Coefficient for D0-T25-4x1.tif: 0.0
Dice Coefficient for D1-T25-10x1.tif: 0.0
Dice Coefficient for D1-T25-20x1.tif: 0.0
Dice Coefficient for D1-T25-40x1.tif: 0.0
Dice Coefficient for D1-T25-4x1.tif: 1.0
Dice Coefficient for D2-T25-10x1.tif: 0.0
Dice Coefficient for D2-T25-20x1.tif: 0.0
Dice Coefficient for D2-T25-40x1.tif: 0.0
Dice Coefficient for D2-T25-4x1.tif: 1.0
Dice Coefficient for D3-T25-10x1.tif: 0.0
Dice Coefficient for D3-T25-20x1.tif: 0.0
Dice Coefficient for D3-T25-40x1.tif: 0.0

Dice coefficients saved to C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\DICE\cellseg_gray.xlsx


This block of code is intended to calculate the Jaccard Index

In [42]:
import os
from PIL import Image
import numpy as np
import pandas as pd

def load_image_as_mask(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    mask = np.array(img)
    return (mask > 128).astype(bool)  # Binarize assuming white=foreground, black=background

def jaccard_index_from_masks(ground_truth_mask, predicted_mask):
    ground_truth_mask = ground_truth_mask.astype(bool)
    predicted_mask = predicted_mask.astype(bool)

    # Resize predicted mask to match ground truth if shapes differ
    if predicted_mask.shape != ground_truth_mask.shape:
        predicted_mask = np.array(
            Image.fromarray(predicted_mask).resize(ground_truth_mask.shape[::-1], Image.NEAREST)
        )

    intersection = np.logical_and(predicted_mask, ground_truth_mask).sum()
    union = np.logical_or(predicted_mask, ground_truth_mask).sum()

    jaccard_index = intersection / union if union > 0 else 1.0
    return jaccard_index

def find_predicted_mask(predicted_folder, filename_base):
    # Check for various common image file extensions
    for ext in ['.tif', '.jpg', '.png']:
        predicted_path = os.path.join(predicted_folder, filename_base + ext)
        if os.path.exists(predicted_path):
            return predicted_path
    return None

def calculate_jaccard_and_save_to_excel(ground_truth_folder, predicted_folder, output_excel):
    jaccard_scores = []

    for filename in os.listdir(ground_truth_folder):
        ground_truth_path = os.path.join(ground_truth_folder, filename)
        filename_base = filename.rsplit('.', 1)[0]  # Base name without extension

        # Load the ground truth mask based on file extension
        if filename.endswith(('.labeling', '.jpg', '.tif')):
            ground_truth_mask = load_image_as_mask(ground_truth_path)
        else:
            print(f"Unsupported file format for {filename}. Skipping...")
            continue

        # Find the predicted mask file with any common image extension
        predicted_path = find_predicted_mask(predicted_folder, filename_base)

        if predicted_path:
            try:
                predicted_mask = load_image_as_mask(predicted_path)

                # Calculate Jaccard Index
                jaccard_score = jaccard_index_from_masks(ground_truth_mask, predicted_mask)
                jaccard_scores.append({'Filename': filename, 'Jaccard Index': jaccard_score})
                print(f"Jaccard Index for {filename}: {jaccard_score}")

            except Exception as e:
                print(f"Error loading predicted mask for {filename}: {e}")
                continue
        else:
            print(f"Predicted mask for {filename} not found in {predicted_folder}")

    # Save results to specified output path
    jaccard_df = pd.DataFrame(jaccard_scores)
    jaccard_df.to_excel(output_excel, index=False)
    print(f"\nJaccard indices saved to {output_excel}")

# Example usage
ground_truth_folder = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Manual Segmentation (Labkit)'
predicted_folder = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Scikit-image (Thresholding)\Gray Scale\Image Files'
output_excel = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\JACCARD\scikit_thr_gray.xlsx'

calculate_jaccard_and_save_to_excel(ground_truth_folder, predicted_folder, output_excel)

Jaccard Index for D0-T25-10x1.tif: 0.0
Jaccard Index for D0-T25-20x1.tif: 0.0
Jaccard Index for D0-T25-40x1.tif: 0.0
Jaccard Index for D0-T25-4x1.tif: 0.0
Jaccard Index for D1-T25-10x1.tif: 0.0
Jaccard Index for D1-T25-20x1.tif: 0.0
Jaccard Index for D1-T25-40x1.tif: 0.0
Jaccard Index for D1-T25-4x1.tif: 0.0
Jaccard Index for D2-T25-10x1.tif: 0.0
Jaccard Index for D2-T25-20x1.tif: 0.0
Jaccard Index for D2-T25-40x1.tif: 0.0
Jaccard Index for D2-T25-4x1.tif: 0.0
Jaccard Index for D3-T25-10x1.tif: 0.0
Jaccard Index for D3-T25-20x1.tif: 0.0
Jaccard Index for D3-T25-40x1.tif: 0.0
Jaccard Index for D3-T25-4x1.tif: 0.0

Jaccard indices saved to C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\JACCARD\scikit_thr_gray.xlsx


This section of the code is meant to calculate the confluency of an image

In [22]:
import os
from PIL import Image
import numpy as np
import pandas as pd

def calculate_confluency(image_path, total_pixels):
    # Load the image and convert to a binary mask (assuming segmentation)
    img = Image.open(image_path).convert('L')
    img_np = np.array(img)

    # Calculate foreground (cell) pixels as non-zero values
    cell_pixels = np.count_nonzero(img_np)
    confluency = (cell_pixels / total_pixels) * 100  # Confluency as a percentage
    return confluency

def calculate_confluency_for_folder(input_folder, total_pixels, output_excel_path):
    confluency_data = []

    # Loop over each image file in the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        # Check if it's an image file (adjust extensions as needed)
        if filename.endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp')):
            try:
                # Calculate confluency for the current image
                confluency = calculate_confluency(file_path, total_pixels)
                confluency_data.append({'Filename': filename, 'Confluency (%)': confluency})
                print(f"Confluency for {filename}: {confluency:.2f}%")
            
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue
        else:
            print(f"Skipping unsupported file format for {filename}")

    # Save confluency data to an Excel file
    confluency_df = pd.DataFrame(confluency_data)
    confluency_df.to_excel(output_excel_path, index=False)
    print(f"\nConfluency values saved to {output_excel_path}")

# Example usage
input_folder = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Scikit-image (Thresholding)\Gray Scale\Image Files'  # Folder containing segmented images
total_pixels = 6487220  # Specify the total pixel count of the images
output_excel_path = r'C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Confluency\scikit_thr_gray_results.xlsx'

calculate_confluency_for_folder(input_folder, total_pixels, output_excel_path)

Confluency for D0-T25-10x1.png: 105.21%
Confluency for D0-T25-20x1.png: 108.77%
Confluency for D0-T25-40x1.png: 97.65%
Confluency for D0-T25-4x1.png: 80.20%
Confluency for D1-T25-10x1.png: 102.40%
Confluency for D1-T25-20x1.png: 100.06%
Confluency for D1-T25-40x1.png: 101.88%
Confluency for D1-T25-4x1.png: 82.35%
Confluency for D2-T25-10x1.png: 93.96%
Confluency for D2-T25-20x1.png: 98.84%
Confluency for D2-T25-40x1.png: 97.15%
Confluency for D2-T25-4x1.png: 73.68%
Confluency for D3-T25-10x1.png: 90.24%
Confluency for D3-T25-20x1.png: 96.36%
Confluency for D3-T25-40x1.png: 97.06%
Confluency for D3-T25-4x1.png: 81.80%

Confluency values saved to C:\Users\nthat\OneDrive - University of Cape Town\Fourth Year\CHE4045Z - RESEARCH\Research Tools\Output\Confluency\scikit_thr_gray_results.xlsx
